In [ ]:
# import the modules
from entity import Entity
from exception import EntityDeleteHasInventoryError
from sql import *
from uuid import uuid4

In [2]:
# try creating an item
item = Entity(description="a lovely plump piglet")
item.id = "1dd566b5-21ba-4f5c-ae90-c1841feab8b5"
print(item.id)
print(item)

1dd566b5-21ba-4f5c-ae90-c1841feab8b5
Lovely Plump Piglet (total inventory: 0)


In [3]:
# return all the entities
entities = select_entity()
print(entities)

[('281d67f9-7236-4c63-adaf-c7fac017d962', 'chads_st_vicarage', 'vicarage', 'st chads', 'st chads vicarage', 'st chads vicarages', 1), ('03dd5c43-d66e-4c3f-a6eb-eb67b47c0373', 'lounge', 'lounge', '', 'lounge', 'lounges', 2), ('468edbe1-74ea-41f2-b1ae-c97806841e0d', 'kitchen', 'kitchen', '', 'kitchen', 'kitchens', 2), ('e5998068-17a4-4c7d-b64f-4b6e28e8b93d', 'dining_room', 'room', 'dining', 'dining room', 'dining rooms', 2), ('4d10faad-8a10-436b-b732-011a7b4ad043', 'chair_yellow', 'chair', 'yellow', 'yellow chair', 'yellow chairs', 0), ('bb85b691-61f2-4a73-8313-3e5367d4b7b6', 'coffee_table', 'table', 'coffee', 'coffee table', 'coffee tables', 0), ('aa5c5230-29d1-4f02-b011-20a6ea03766a', 'toaster', 'toaster', '', 'toaster', 'toasters', 0), ('f03c6766-6984-4821-9dcf-a1efc231758f', 'coaster', 'coaster', '', 'coaster', 'coasters', 0), ('3e874dbd-0cfc-4388-ad8b-ddfcc1b6a329', 'bread_of_slice', 'bread', 'slice of', 'slice of bread', 'slices of bread', 0), ('a2c768d1-372b-4c52-94f6-cdab9a297b7e

In [4]:
# return the new entity
piggy = select_entity(entity_key="lovely_piglet_plump")
print(piggy)

('a2c768d1-372b-4c52-94f6-cdab9a297b7e', 'lovely_piglet_plump', 'piglet', 'lovely plump', 'lovely plump piglet', 'lovely plump piglets', 0)


In [5]:
# update the entity and check the description in the database has updated
item2 = Entity("a plump lovely piglet")
print(item2.singular)
piggy2 = select_entity(entity_key="lovely_piglet_plump")
print(piggy2)

plump lovely piglet
('a2c768d1-372b-4c52-94f6-cdab9a297b7e', 'lovely_piglet_plump', 'piglet', 'plump lovely', 'plump lovely piglet', 'plump lovely piglets', 0)


In [6]:
inventory = select_inventory()
for inv in inventory:
    print(dict(inv))

{'inventory_id': '8ce8d635-c5c5-49d7-bdcf-2b5cfb2298fe', 'inventory_id_parent': '8ce8d635-c5c5-49d7-bdcf-2b5cfb2298fe', 'entity_id_parent': '281d67f9-7236-4c63-adaf-c7fac017d962', 'entity_id_child': '03dd5c43-d66e-4c3f-a6eb-eb67b47c0373', 'quantity': 1, 'hierarchy_level': 1, 'created_time': '2025-02-19 16:04:23'}
{'inventory_id': '4dbba35f-ef8f-4af1-9148-02f2a0e1cbd1', 'inventory_id_parent': '4dbba35f-ef8f-4af1-9148-02f2a0e1cbd1', 'entity_id_parent': '281d67f9-7236-4c63-adaf-c7fac017d962', 'entity_id_child': '468edbe1-74ea-41f2-b1ae-c97806841e0d', 'quantity': 1, 'hierarchy_level': 1, 'created_time': '2025-02-19 16:04:23'}
{'inventory_id': '6dc856f4-42fe-4fe0-868b-9d002eff2e27', 'inventory_id_parent': '6dc856f4-42fe-4fe0-868b-9d002eff2e27', 'entity_id_parent': '281d67f9-7236-4c63-adaf-c7fac017d962', 'entity_id_child': 'e5998068-17a4-4c7d-b64f-4b6e28e8b93d', 'quantity': 1, 'hierarchy_level': 1, 'created_time': '2025-02-19 16:04:23'}
{'inventory_id': '2dfea04c-e8ea-4735-8b9a-4a9bf92dca08'

In [7]:
# add some inventory for the entity
insert_inventory(
    inventory_id = "6d652592-7301-4819-9660-3c5ea40f0b50",
    inventory_id_parent = "8ce8d635-c5c5-49d7-bdcf-2b5cfb2298fe", # loung in vicarage
    entity_id_parent = "03dd5c43-d66e-4c3f-a6eb-eb67b47c0373", # lounge
    entity_id_child = "1dd566b5-21ba-4f5c-ae90-c1841feab8b5", # piglet
    quantity = 10,
)
print(item)


Lovely Plump Piglet (total inventory: 10)


In [8]:
# delete the entities !!!Should fail with the EntityDeleteHasInventoryError!
print(item)
try:
    item.delete_existing()
except EntityDeleteHasInventoryError:
    print("Item can't be deleted as has inventory")       

Lovely Plump Piglet (total inventory: 10)
Item can't be deleted as has inventory


In [9]:
# remove the inventory so item can be deleted
delete_inventory("6d652592-7301-4819-9660-3c5ea40f0b50")
item.delete_existing()
piggy = select_entity(entity_key="lovely_piglet_plump")
print(item)
print(piggy)


Lovely Plump Piglet (total inventory: 0)
None
